<h3> Macierz masy P1</h3> sposób1

In [ ]:
def mass_matrix_calculate(P, T, K):
    """    
    Funkcja obliczająca macierz masy dla P1, z gotowej formuły.
    input: P - np.array(), lista węzłów; T - np.array(), connection matrix;
    K - float, objetosc elementu siatki jednorodnej.
    output: M - scipy.sparse.csr_matrix, macierz masy
    """

    n = len(P)  # liczba węzłów 
    nt = T[0].size  # liczba elementów siatki
    #M = scipy.sparse.lil_matrix((n, n), dtype=float)  # macierz masy o wymiarze n x n
    size = 9 * nt  # liczymy długości tablic dla formatu csr
    data = np.zeros([size])
    row = np.zeros([size])
    col = np.zeros([size])
    ind = 0
    # K stałe to wystarczy M_k policzyć tylko raz
    
    M_k = 1 / 12 * np.mat([[2, 1, 1], [1, 2, 1], [1, 1, 2]]) * K  # lokalna macierz masy

    # Local to global mapping
    
    for k in range(nt):  # ilość elementów
        for i in range(3):
            for j in range(3):
                data[ind] = M_k[i, j]
                row[ind] = T[i,k]
                col[ind] = T[j,k]
                ind = ind + 1
    M = scipy.sparse.csr_matrix((data, (row, col)), shape=(n, n))
    
    return M


<h3> Macierz masy P1 </h3> sposób2

In [ ]:
def P1_mass_matrix_calculate(P, T):
    """
    funkcja obliczająca macierz masy P2, przy uzyciu mapowania afinicznego.
    input: P - np.array(), macierz węzłów P2; connections - np.array(), macierz połączeń P2.
    output: M - scipy.sparse.csr_matrix, macierz masy P2
    """

    qwgts, rspts = Gausspoints(4)  # wagi i punkty dla kwadratury Gaussa
    n = len(P)  # liczba węzłów
    nt = T.shape[1]  # liczba elementów siatki
    size = 9*nt  # liczymy długości tablic dla formatu csr
    data = np.zeros([size])
    row = np.zeros([size])
    col = np.zeros([size])
    ind = 0
    
    for i in range(nt):  # iterujemy dla każdego trójkąta
        local_nodes = T[:, i]  # zgarniamy węzły trójkąta
        M_k = np.zeros([3, 3])  # rezerwujemy miejsce na lokalną macierz masy

        for q in range(len(qwgts)):  # pętla kwadatury -
            # 1 iteracja dodajemy 1 element sumy kwadratury dla lokalnej macierzy masy
            # Zgarniamy współrzędne kolejnego punktu, w którym liczymy kwadraturę
            r = rspts[0, q]
            s = rspts[1, q]
            # Liczymy komponenty mapowania izoparametrycznego
            S, dSdx, dSdy, detJ = isoparametric_mapping(r, s, P[local_nodes, :])
            wxarea = qwgts[q] * detJ / 2  # liczymy stałą ( waga * pole trójkąta )
            # Liczymy lokalną macierz masy
            S = S[np.newaxis]  # dodajemy wymiar, aby móc transponować
            M_k = M_k + np.matmul(S.T, S) * wxarea

        # Local to global mapping

        for b in range(3):
            for c in range(3):
                data[ind] = M_k[b, c]
                row[ind] = local_nodes[b]
                col[ind] = local_nodes[c]
                ind = ind + 1
    M = scipy.sparse.csr_matrix((data, (row, col)), shape=(n, n))
    return M


<h3> Macierz masy P2 </h3>

In [ ]:
def P2_mass_matrix_calculate(P, T):
    """
    funkcja obliczająca macierz masy P2, przy uzyciu mapowania afinicznego.
    input: P - np.array(), macierz węzłów P2; connections - np.array(), macierz połączeń P2.
    output: M - scipy.sparse.csr_matrix, macierz masy P2
    """

    qwgts, rspts = Gausspoints(4)  # wagi i punkty dla kwadratury Gaussa
    n = len(P)  # liczba węzłów
    nt = T.shape[1]  # liczba elementów siatki
    size = 36*nt  # liczymy długości tablic dla formatu csr
    data = np.zeros([size])
    row = np.zeros([size])
    col = np.zeros([size])
    ind = 0
    
    for i in range(nt):  # iterujemy dla każdego trójkąta
        local_nodes = T[:, i]  # zgarniamy węzły trójkąta
        M_k = np.zeros([6, 6])  # rezerwujemy miejsce na lokalną macierz masy

        for q in range(len(qwgts)):  # pętla kwadatury -
            # 1 iteracja dodajemy 1 element sumy kwadratury dla lokalnej macierzy masy
            # Zgarniamy współrzędne kolejnego punktu, w którym liczymy kwadraturę
            r = rspts[0, q]
            s = rspts[1, q]
            # Liczymy komponenty mapowania izoparametrycznego
            S, dSdx, dSdy, detJ = isoparametric_mapping(r, s, P[local_nodes, :])
            wxarea = qwgts[q] * detJ / 2  # liczymy stałą ( waga * pole trójkąta )
            # Liczymy lokalną macierz masy
            S = S[np.newaxis]  # dodajemy wymiar, aby móc transponować
            M_k = M_k + np.matmul(S.T, S) * wxarea

        # Local to global mapping
        for b in range(6):
            for c in range(6):
                data[ind] = M_k[b, c]
                row[ind] = local_nodes[b]
                col[ind] = local_nodes[c]
                ind = ind + 1
    M = scipy.sparse.csr_matrix((data, (row, col)), shape=(n, n))
    return M


<h3> Macierz masy P2 </h3> siatka jednorodna

In [ ]:
def P2_mass_matrix_uniform_mesh(P, T):
    """
    funkcja obliczająca macierz masy P2, przy uzyciu mapowania afinicznego dla siatki jednorodnej.
    input: P - np.array(), macierz węzłów P2; connections - np.array(), macierz połączeń P2.
    output: M - scipy.sparse.csr_matrix, macierz masy P2
    """

    qwgts, rspts = Gausspoints(4)  # wagi i punkty dla kwadratury Gaussa
    n = len(P)  # liczba węzłów
    nt = T.shape[1]  # liczba elementów siatki
    size = 36*nt  # liczymy długości tablic dla formatu csr
    data = np.zeros([size])
    row = np.zeros([size])
    col = np.zeros([size])
    ind = 0
    
    for i in range(nt):  # iterujemy dla każdego trójkąta
        local_nodes = T[:, i]  # zgarniamy węzły trójkąta
        M_k = np.zeros([6, 6])  # rezerwujemy miejsce na lokalną macierz masy
        if i == 0:
            for q in range(len(qwgts)):  # pętla kwadatury -
                # 1 iteracja dodajemy 1 element sumy kwadratury dla lokalnej macierzy masy
                # Zgarniamy współrzędne kolejnego punktu, w którym liczymy kwadraturę
                r = rspts[0, q]
                s = rspts[1, q]
                # Liczymy komponenty mapowania izoparametrycznego
                S, dSdx, dSdy, detJ = isoparametric_mapping(r, s, P[local_nodes, :])
                wxarea = qwgts[q] * detJ / 2  # liczymy stałą ( waga * pole trójkąta )
                # Liczymy lokalną macierz masy
                S = S[np.newaxis]  # dodajemy wymiar, aby móc transponować
                M_k = M_k + np.matmul(S.T, S) * wxarea

        # Local to global mapping
        for b in range(6):
            for c in range(6):
                data[ind] = M_k[b, c]
                row[ind] = local_nodes[b]
                col[ind] = local_nodes[c]
                ind = ind + 1
    M = scipy.sparse.csr_matrix((data, (row, col)), shape=(n, n))
    return M


<h3> Macierz sztywności P1</h3>
sposób1

In [ ]:
def stiffness_matrix_calculate(P, T, K):
    """
    funkcja obliczająca macierz sztywności dla P1, z gotowej formuły.
    input: input: P - np.array(), lista węzłów; T - np.array(), connection matrix;
    K - float, objetosc elementu siatki jednorodnej
    output: A - scipy.sparse.csr_matrix, macierz sztywności P1
    """

    n = len(P)  # liczba węzłów
    nt = T[0].size  # liczba elementów siatki
    #A = scipy.sparse.lil_matrix((n, n), dtype=float)  # macierz sztywności o wymiarze n x n
    size = 9*nt  # liczymy długości tablic dla formatu csr
    data = np.zeros([size])
    row = np.zeros([size])
    col = np.zeros([size])
    ind = 0
    
    for k in range(nt):

        # Obliczamy potrzebne współczynniki funkcji kształtu dla z-tego trójkąta
        x = np.array(P[T[:, k]])
        b = np.array([(x[1][1] - x[2][1]), (x[2][1] - x[0][1]), (x[0][1] - x[1][1])]) / (2 * K)
        c = np.array([(x[2][0] - x[1][0]), (x[0][0] - x[2][0]), (x[1][0] - x[0][0])]) / (2 * K)
        a_ = 1

        # Obliczamy lokalną macierz sztywności

        A_k = a_ * np.mat([[b[0] ** 2 + c[0] ** 2, b[0] * b[1] + c[0] * c[1], b[0] * b[2] + c[0] * c[2]],
                           [b[1] * b[0] + c[1] * c[0], b[1] ** 2 + c[1] ** 2, b[1] * b[2] + c[1] * c[2]],
                           [b[2] * b[0] + c[2] * c[0], b[2] * b[1] + c[2] * c[1], b[2] ** 2 + c[2] ** 2]]) * K

        # Local to global mapping

        for i in range(3):
            for j in range(3):
                data[ind] = A_k[i, j]
                row[ind] = T[i,k]
                col[ind] = T[j,k]
                ind = ind + 1
    A = scipy.sparse.csr_matrix((data, (row, col)), shape=(n, n))

    return A


<h3> Maciesz sztywności P1 </h3>
sposób2

In [ ]:
def P1_stiffness_matrix_calculate(P,T):
    """
    funkcja obliczająca macierz sztywności dla P2, przy użyciu mapowania afinicznego.
    input: input: P - np.array(), lista węzłów; T - np.array(), connection matrix;
    output: A - scipy.sparse.csr_matrix, macierz sztywności P2
    """
    
    qwgts,rspts = Gausspoints(4) # wagi i punkty dla kwadratury Gaussa
    n = P.shape[0] # ilość węzłów
    nt = T.shape[1] # ilość elementów siatki
    #A = scipy.sparse.lil_matrix((n,n),dtype=float) # rezerwujemy miejsce na macierz sztywności
    size = 9*nt  # liczymy długości tablic dla formatu csr
    data = np.zeros([size])
    row = np.zeros([size])
    col = np.zeros([size])
    ind = 0
    
    for i in range(nt): # iterujemy dla każdego trójkąta
        local_nodes = T[:,i] # zgarniamy węzły trójkąta
        A_k = np.zeros([3,3]) # rezerwujemy miejsce na lokalną macierz sztywności
        
        for q in range(len(qwgts)):  # pętla kwadatury -
            # 1 iteracja dodajemy 1 element sumy kwadratury dla lokalnej macierzy sztywności
            # Zgarniamy współrzędne kolejnego punktu, w którym liczymy kwadraturę
            r = rspts[0, q]
            s = rspts[1, q]
            # Liczymy komponenty mapowania izoparametrycznego
            S, dSdx, dSdy, detJ = isoparametric_mapping(r, s, P[local_nodes])
            wxarea = qwgts[q] * detJ / 2  # liczymy stałą ( waga * pole trójkąta )
            # Liczymy lokalną macierz sztywności
            dSdx = dSdx[np.newaxis] # dodajemy wymiar, aby móc transponować
            dSdy = dSdy[np.newaxis]
            A_k = A_k + (np.matmul(dSdx.T, dSdx) + np.matmul(dSdy.T, dSdy)) * wxarea

        # Local to global mapping    
        
        for b in range(3):
            for c in range(3):
                data[ind] = A_k[b, c]
                row[ind] = local_nodes[b]
                col[ind] = local_nodes[c]
                ind = ind + 1
    A = scipy.sparse.csr_matrix((data, (row, col)), shape=(n, n))
                
    return A


<h3> Macierz sztywności P2 </h3>

In [ ]:
def P2_stiffness_matrix_calculate(P,T):
    """
    funkcja obliczająca macierz sztywności dla P2, przy użyciu mapowania afinicznego.
    input: input: P - np.array(), lista węzłów; T - np.array(), connection matrix;
    output: A - scipy.sparse.csr_matrix, macierz sztywności P2
    """
    
    qwgts,rspts = Gausspoints(4) # wagi i punkty dla kwadratury Gaussa
    n = P.shape[0] # ilość węzłów
    nt = T.shape[1] # ilość elementów siatki
    size = 36*nt  # liczymy długości tablic dla formatu csr
    data = np.zeros([size])
    row = np.zeros([size])
    col = np.zeros([size])
    ind = 0

    for i in range(nt): # iterujemy dla każdego trójkąta
        local_nodes = T[:,i] # zgarniamy węzły trójkąta
        A_k = np.zeros([6,6]) # rezerwujemy miejsce na lokalną macierz sztywności
        
        for q in range(len(qwgts)):  # pętla kwadatury -
            # 1 iteracja dodajemy 1 element sumy kwadratury dla lokalnej macierzy sztywności
            # Zgarniamy współrzędne kolejnego punktu, w którym liczymy kwadraturę
            r = rspts[0, q]
            s = rspts[1, q]
            # Liczymy komponenty mapowania izoparametrycznego
            S, dSdx, dSdy, detJ = isoparametric_mapping(r, s, P[local_nodes])
            wxarea = qwgts[q] * detJ / 2  # liczymy stałą ( waga * pole trójkąta )
            # Liczymy lokalną macierz sztywności
            dSdx = dSdx[np.newaxis] # dodajemy wymiar, aby móc transponować
            dSdy = dSdy[np.newaxis]
            A_k = A_k + (np.matmul(dSdx.T, dSdx) + np.matmul(dSdy.T, dSdy)) * wxarea

        # Local to global mapping    
        for b in range(6):
            for c in range(6):
                data[ind] = A_k[b, c]
                row[ind] = local_nodes[b]
                col[ind] = local_nodes[c]
                ind = ind + 1
    A = scipy.sparse.csr_matrix((data, (row, col)), shape=(n, n))
                
    return A


<h3> Macierz sztywności P2 </h3>
siatka jednorodna

In [ ]:
def P2_stiffness_matrix_uniform_mesh(P,T):
    """
    funkcja obliczająca macierz sztywności dla P2, przy użyciu afinicznego dla siatki jednorodnej.
    input: input: P - np.array(), lista węzłów; T - np.array(), connection matrix;
    output: A - scipy.sparse.csr_matrix, macierz sztywności P2
    """
    
    qwgts,rspts = Gausspoints(4) # wagi i punkty dla kwadratury Gaussa
    n = P.shape[0] # ilość węzłów
    nt = T.shape[1] # ilość elementów siatki
    size = 36*nt  # liczymy długości tablic dla formatu csr
    data = np.zeros([size])
    row = np.zeros([size])
    col = np.zeros([size])
    ind = 0

    def local_stiffness():
        A_k = np.zeros([6,6]) # rezerwujemy miejsce na lokalną macierz sztywności
        for q in range(len(qwgts)):  # pętla kwadatury -
            # 1 iteracja dodajemy 1 element sumy kwadratury dla lokalnej macierzy sztywności
            # Zgarniamy współrzędne kolejnego punktu, w którym liczymy kwadraturę
            r = rspts[0, q]
            s = rspts[1, q]
            # Liczymy komponenty mapowania izoparametrycznego
            S, dSdx, dSdy, detJ = isoparametric_mapping(r, s, P[local_nodes])
            wxarea = qwgts[q] * detJ / 2  # liczymy stałą ( waga * pole trójkąta )
            # Liczymy lokalną macierz sztywności
            dSdx = dSdx[np.newaxis] # dodajemy wymiar, aby móc transponować
            dSdy = dSdy[np.newaxis]
            A_k = A_k + (np.matmul(dSdx.T, dSdx) + np.matmul(dSdy.T, dSdy)) * wxarea
        return A_k
    
    def local2global(A_k, ind):
        for b in range(6):
            for c in range(6):
                data[ind] = A_k[b, c]
                row[ind] = local_nodes[b]
                col[ind] = local_nodes[c]
                ind = ind + 1
        return ind
        
    for i in range(nt): # iterujemy dla każdego trójkąta
        local_nodes = T[:,i] # zgarniamy węzły trójkąta

        if i == 0:
            A_keven = local_stiffness()
            ind = local2global(A_keven, ind)
        elif i == 1:
            A_kodd = local_stiffness()
            ind = local2global(A_kodd, ind)
        elif i % 2 == 0:
            ind = local2global(A_keven, ind)
        else:
            ind = local2global(A_kodd, ind)
        # Local to global mapping    
    A = scipy.sparse.csr_matrix((data, (row, col)), shape=(n, n))
                
    return A


<h3>Podmacierz dywergencji </h3>


In [ ]:
def divergence_matrix_calculate(nodesP1, nodesP2, connectionsP1, connectionsP2, derivative):
    """
    Funkcja obliczająca podmacierz dywergencji.
    input: nodesP1 i nodesP2 - np.array(), np.array(), macierz węzłów odpowiednio P1 i P2,
           connectionsP1 i coneectionsP2 - np.array(), np.array(), macierz połączeń odpowiednio P1 i P2
    output: D - scipy.sparse.csr_matrix, macierz dywergencji
    """

    qwgts, rspts = Gausspoints(4)  # wagi i punkty dla kwadratury Gaussa
    nP1 = nodesP1.shape[0]  # ilość węzłów P1
    nP2 = nodesP2.shape[0]  # ilość węzłów P2
    nt = connectionsP1.shape[1]  # ilość elementów siatki P1
    size = 3*6*nt  # liczymy długości tablic dla formatu csr
    data = np.zeros([size])
    row = np.zeros([size])
    col = np.zeros([size])
    ind = 0
    
    for i in range(nt):  # iterujemy dla każdego trójkąta
        local_nodesP1 = connectionsP1[:, i]  # zgarniamy węzły P1
        local_nodesP2 = connectionsP2[:, i]  # zgarniamy węzły P2
        D_k = np.zeros([6, 3])  # rezerwujemy miejsce na lokalną macierz dywergencji

        for q in range(len(qwgts)):  # pętla kwadatury -
            # 1 iteracja dodajemy 1 element sumy kwadratury dla lokalnej macierzy sztywności
            # Zgarniamy współrzędne kolejnego punktu, w którym liczymy kwadraturę
            r = rspts[0, q]
            s = rspts[1, q]
            # Liczymy komponenty mapowania izoparametrycznego
            resultP1 = isoparametric_mapping(r, s, nodesP1[local_nodesP1, :])
            S_P1 = resultP1[0]
            S_P1 = S_P1[np.newaxis]
            
            #S_P2, dSdx_P2, dSdy_P2, detJ = isoparametric_mapping(r, s, nodesP2[local_nodesP2, :])
            P2mapping_results = isoparametric_mapping(r, s, nodesP2[local_nodesP2, :])
            detJ = P2mapping_results[3]
            if derivative == 'x':
                dS = P2mapping_results[1]
            elif derivative == 'y':
                dS = P2mapping_results[2]
                
            wxarea = qwgts[q] * detJ / 2 # liczymy stałą ( waga * pole trójkąta )
            dS = dS[np.newaxis]  # dodajemy wymiar, aby móc transponować
            D_k = D_k + np.matmul(dS.T, S_P1) * wxarea
            
        # Local to global mapping
        for b in range(6):
            for c in range(3):
                data[ind] = D_k[b, c]
                row[ind] = local_nodesP2[b]
                col[ind] = local_nodesP1[c]
                ind = ind + 1
    D = scipy.sparse.csr_matrix((data, (row, col)), shape=(nP2, nP1))
    return D


<h3> Podmacierz dywergencji</h3>
siatka jednorodna

In [ ]:
def divergence_matrix_uniform_mesh(nodesP1, nodesP2, connectionsP1, connectionsP2, derivative):
    """
    Funkcja obliczająca macierz dywergencji dla siatki jednorodnej.
    input: nodesP1 i nodesP2 - np.array(), np.array(), macierz węzłów odpowiednio P1 i P2,
           connectionsP1 i coneectionsP2 - np.array(), np.array(), macierz połączeń odpowiednio P1 i P2
    output: D - scipy.sparse.csr_matrix, macierz dywergencji
    """

    qwgts, rspts = Gausspoints(4)  # wagi i punkty dla kwadratury Gaussa
    nP1 = nodesP1.shape[0]  # ilość węzłów P1
    nP2 = nodesP2.shape[0]  # ilość węzłów P2
    nt = connectionsP1.shape[1]  # ilość elementów siatki P1
    size = 3*6*nt  # liczymy długości tablic dla formatu csr
    data = np.zeros([size])
    row = np.zeros([size])
    col = np.zeros([size])
    ind = 0
    
    def local_divergence():
        D_k = np.zeros([6, 3])  # rezerwujemy miejsce na lokalną macierz dywergencji
        for q in range(len(qwgts)):  # pętla kwadatury -
            # 1 iteracja dodajemy 1 element sumy kwadratury dla lokalnej macierzy sztywności
            # Zgarniamy współrzędne kolejnego punktu, w którym liczymy kwadraturę
            r = rspts[0, q]
            s = rspts[1, q]
            # Liczymy komponenty mapowania izoparametrycznego
            resultP1 = isoparametric_mapping(r, s, nodesP1[local_nodesP1, :])
            S_P1 = resultP1[0]
            S_P1 = S_P1[np.newaxis]
            
            #S_P2, dSdx_P2, dSdy_P2, detJ = isoparametric_mapping(r, s, nodesP2[local_nodesP2, :])
            P2mapping_results = isoparametric_mapping(r, s, nodesP2[local_nodesP2, :])
            detJ = P2mapping_results[3]
            if derivative == 'x':
                dS = P2mapping_results[1]
            elif derivative == 'y':
                dS = P2mapping_results[2]
                
            wxarea = qwgts[q] * detJ / 2 # liczymy stałą ( waga * pole trójkąta )
            dS = dS[np.newaxis]  # dodajemy wymiar, aby móc transponować
            D_k = D_k + np.matmul(dS.T, S_P1) * wxarea
        return D_k
    
    def local2global(D_k, ind):
        for b in range(6):
            for c in range(3):
                data[ind] = D_k[b, c]
                row[ind] = local_nodesP2[b]
                col[ind] = local_nodesP1[c]
                ind = ind + 1
        return ind
    
    for i in range(nt):  # iterujemy dla każdego trójkąta
        local_nodesP1 = connectionsP1[:, i]  # zgarniamy węzły P1
        local_nodesP2 = connectionsP2[:, i]  # zgarniamy węzły P2
        if i == 0:
            D_keven = local_divergence()
            ind = local2global(D_keven, ind)
        elif i == 1:
            D_kodd = local_divergence()
            ind = local2global(D_kodd, ind)
        elif i % 2 == 0:
            ind = local2global(D_keven, ind)
        else:
            ind = local2global(D_kodd, ind)
                
    D = scipy.sparse.csr_matrix((data, (row, col)), shape=(nP2, nP1))
    return D


<h3>Wektor obciążeń P1</h3> sposób1

In [10]:
def load_vector_calculate(P, T, force, K):
    """
    Funkcja obliczająca wektor obciążeń, z gotowej formuły.
    input: P - np.array(), macierz węzłów; T - np.array(), macierz połączeń;
           force - np.array(), macierz sił ( wektory sił dla każdego momentu q );
           K - float, pole elementu
    output: b - np.array(), macierz obciążeń ( wektory obciążeń dla każego momentu q )
    """

    n = P.shape[0]  # ilość węzłów
    nt = T[0].size  # ilość elementów
    b = np.zeros(n)  # rezerwujemy miejsce na wektor obciążeń

    # Iterujemy dla każdego momentu q

    for k in range(nt):

        b_k = 1 / 3 * np.array([force[T[0, k]], force[T[1, k]],
                                force[T[2, k]]]) * K

        # Local to global mapping

        for i in range(3):
            b[T[i, k]] = b[T[i, k]] + b_k[i]
    return b


<h3>Wektor obciążeń P1</h3> sposób2

In [ ]:
def P1_load_vector_calculate(P,T,force):
    """
    Funkcja obliczająca wektor obciążeń P2, wykorzystując mapowanie afiniczne.
    input: P - np.array(), macierz węzłów; T - np.array(), macierz połączeń;
           force - np.array(), macierz sił ( wektory sił dla każdego momentu q );
    output: b - np.array(), macierz obciążeń ( wektory obciążeń dla każego momentu q )
    """
    
    qwgts, rspts = Gausspoints(4)  # wagi i punkty dla kwadratury Gaussa
    n = P.shape[0]  # ilość węzłów
    nt = T.shape[1]  # ilość elementów siatki
    b = np.zeros([n, force.shape[1]])  # rezerwujemy miejsce na wektor obciążeń

    for t in range(force.shape[1]):
        for i in range(nt): # iterujemy dla każdego trójkąta
            local_nodes = T[:,i] # zgarniamy węzły trójkąta
            b_k = np.zeros([1,3])
            for q in range(len(qwgts)):  # pętla kwadatury -
                # 1 iteracja dodajemy 1 element sumy kwadratury dla lokalnej macierzy sztywności
                # Zgarniamy współrzędne kolejnego punktu, w którym liczymy kwadraturę
                r = rspts[0, q]
                s = rspts[1, q]
                # Liczymy komponenty mapowania izoparametrycznego
                S, dSdx, dSdy, detJ = isoparametric_mapping(r, s, P[local_nodes])
                wxarea = qwgts[q] * detJ / 2  # liczymy stałą ( waga * pole trójkąta )
                # Liczymy lokalny wektor obciążeń
                b_k = b_k + np.multiply(force[local_nodes, t], S) * wxarea
                
            # Local to global mapping

            for j in range(3):
                b[T[j, i], t] = b[T[j, i], t] + b_k[0, j]

    return b


<h3>Wektor obciążeń P2</h3>

In [ ]:
def P2_load_vector_calculate(P,T,force):
    """
    Funkcja obliczająca wektor obciążeń P2, wykorzystując mapowanie afiniczne.
    input: P - np.array(), macierz węzłów; T - np.array(), macierz połączeń;
           force - np.array(), macierz sił ( wektory sił dla każdego momentu q );
    output: b - np.array(), macierz obciążeń ( wektory obciążeń dla każego momentu q )
    """
    
    qwgts, rspts = Gausspoints(4)  # wagi i punkty dla kwadratury Gaussa
    n = P.shape[0]  # ilość węzłów
    nt = T.shape[1]  # ilość elementów siatki
    b = np.zeros([n, force.shape[1]])  # rezerwujemy miejsce na wektor obciążeń

    for t in range(force.shape[1]):
        for i in range(nt): # iterujemy dla każdego trójkąta
            local_nodes = T[:,i] # zgarniamy węzły trójkąta
            b_k = np.zeros([1,6])
            for q in range(len(qwgts)):  # pętla kwadatury -
                # 1 iteracja dodajemy 1 element sumy kwadratury dla lokalnej macierzy sztywności
                # Zgarniamy współrzędne kolejnego punktu, w którym liczymy kwadraturę
                r = rspts[0, q]
                s = rspts[1, q]
                # Liczymy komponenty mapowania izoparametrycznego
                S, dSdx, dSdy, detJ = isoparametric_mapping(r, s, P[local_nodes])
                wxarea = qwgts[q] * detJ / 2  # liczymy stałą ( waga * pole trójkąta )
                # Liczymy lokalny wektor obciążeń
                b_k = b_k + np.multiply(force[local_nodes, t], S) * wxarea
                
            # Local to global mapping

            for j in range(6):
                b[T[j, i], t] = b[T[j, i], t] + b_k[0, j]

    return b


<h3> Warunki brzegowe </h3>

In [ ]:
def set_boundaries(obstacle, X, Y, P, dirichlet_z, dirichlet_f):
    """
    Funkcja do macierzy C obszaru reprezentującej obszar w oparciu o węzły wkleja:
    -1 jeśli wierzchołek dotyczy warunku dirichleta u = g_D, 0 jeśli warunek dirichleta u = 0.
    input:  obstacle - np.array(), lista z węzłami definiującymi przeszkody,
            X - tablica, będąca podziałką na osi x wg. szerokości węzła;
            Y - tablica, będąca podziałką na osi y wg. wysokości węzła;
            P - lista węzłów; dirichlet_z - lista brzegu gdzie u = 0;
            dirichlet_f - lista brzegu gdzie g_D = 0.
    output: dirichlet_zero_nodes, dirichlet_fun_nodes-
            listy z węzłami odnoszącymi się do warunków brzegowych;
            C - np.array(), macierz reprezentacji obszaru;
            no_boundary - list(), lista węzłów bez warunku brzegowego.
            
    """
    
    dirichlet_zero_nodes = []  # listy węzłów dla odpowiednich warunków brzegowych
    dirichlet_fun_nodes = []
    no_boundary = []
    
    x_gap = (X[1]) - (X[0])  # obliczamy odstępy na osiach
    y_gap = (Y[1]) - (Y[0])

    C = np.ones((len(Y), len(X)))  # utworzenie tablicy jedynek,
    # reprezentującej obszar bez przeszkody, poniżej odpowiednio ją uzupełniamy
    
    if 'left' in dirichlet_f:
        C[:, 0] = -1  # tu będzie drichlet u = g_D
    elif 'left' in dirichlet_z:
        C[:, 0] = 0  # tu dirichlet u = 0
    if 'right' in dirichlet_f:
        C[:, -1] = -1  # tu będzie drichlet u = g_D
    elif 'right' in dirichlet_z:
        C[:, -1] = 0  # tu dirichlet u = 0 
    if 'top' in dirichlet_f:
        C[-1, :] = -1  # tu będzie drichlet u = g_D
    elif 'top' in dirichlet_z:
        C[-1, :] = 0  # tu dirichlet u = 0 
    if 'bottom' in dirichlet_f:
        C[0, :] = -1  # tu będzie drichlet u = g_D
    elif 'bottom' in dirichlet_z:
        C[0, :] = 0  # tu dirichlet u = 0 

    # Znajdujemy współrzędne przeszkód
                
    myobstacle = np.zeros((len(obstacle), 4), int)  # tablica współrzędnych przeszkód
        
    for j in range(len(myobstacle)):
        myobstacle[j, 0] = int(np.around((obstacle[j][0][0] - X[0]) / x_gap,0))
        myobstacle[j, 1] = int(np.around((obstacle[j][1][0] - X[0]) / x_gap,0))
        myobstacle[j, 2] = int(np.around((obstacle[j][0][1] - Y[0]) / y_gap,0))
        myobstacle[j, 3] = int(np.around((obstacle[j][2][1] - Y[0]) / y_gap,0))

    # Dla każdej przeszkody rozważamy warunek brzegowy
    
    for k in range(len(myobstacle)):

        object_left = myobstacle[k][0]
        object_right = myobstacle[k][1]
        object_down = myobstacle[k][2]
        object_up = myobstacle[k][3]
        C[object_down:object_up + 1, object_left:object_right + 1] = 0  # wklejenie 0 w miejsce przeszkody + jej brzeg

    # Przydzielamy węzły do warunków brzegowych
        
    for k in range(P.shape[0]):
        j = int(np.around((P[k, 0]-X[0])/x_gap,0))  # obliczamy indeks j
        i = int(np.around((P[k, 1]-Y[0])/y_gap,0))  # obliczamy indeks i
        if C[i][j] == 0:
            if dirichlet_z:
                dirichlet_zero_nodes.append(k)
        elif C[i][j] == -1:

            if dirichlet_f:
                dirichlet_fun_nodes.append(k)
        else:
            no_boundary.append(k)
            
    return C, dirichlet_zero_nodes, dirichlet_fun_nodes, no_boundary


In [ ]:
def dirichlet_function(y, option=1):
    """
        Funkcja obliczająca wynik funkcji g_D w punkcie.
        input: y - float, druga współrzędna węzła; option - int, wybór funkcji g_D.
        return: wynik funkcji g_D w punkcie.
    """
    
    if option == 1: 
        return -(y-wall_bottom)*(y-wall_top)
    elif option == 2:
        return -(y-wall_bottom)*(y-wall_top)**2*(y-(wall_top-wall_bottom)/2)
    elif option == 3:
        return -1.2*(y-wall_bottom)*(y-wall_top)/wall_top**2


In [ ]:
def force_vector(size, option=1):
    """
        Funkcja wyboru wektora sił f.
        input: size - int > 0, wielkość wektora; option - int, wybór rodzaju wektora sił.
        output: wektor sił 
    """
    
    if option == 1: 
        return np.zeros([size,1])
    elif option == 2:
        return np.ones([size,1])
    elif option == 3:
        return np.ones([size,1])*5
    elif option == 4:
        return np.ones([size,1])*10
    elif option == 5:
        return np.ones([size,1])*50
    elif option == 6:
        return np.ones([size,1])*100
        

In [6]:
def solver_stokes_stationary(A, D_x, D_y, b, ksi_0, dirichlet_zero_nodes, dirichlet_fun_nodes, no_boundary, P):
    """
        Solver problemu stacjonarnego stokesa.
        input:  A - scipy.sparse.csr_matrix(), macierz sztywności;
                D_x, D_y - scipy.sparse.csr_matrix(), podmacierze dywergencji;
                ksi_0 - np.array(), wektor wartości początkowych;
                b - np.array(), wektor obciążeń;
                dirichlet_zero_nodes - list(), lista węzłów dla warunku brzegowego Dirichleta u = 0;
                dirichlet_fun_nodes - list(), lista węzłów dla warunku brzegowego Dirichleta u = g_D;
                no_boundary - list(), lista węzłów bez warunku brzegowego;
                P - np.array(), tablica węzłów.
        output: ksi_x - np.array(), wektor rozwiązań u_1;
                ksi_y - np.array(), wektor rozwiązań u_2;
                gamma - np.array(), wektor rozwiązań p;
        

    """
    
    t1 = time.time()
    n_u_x = D_x.shape[0]  # zgarniamy długość wektora rozwiązań prędkości po wsp. x
    n_u_y = n_u_x  # po wsp. y jest mu równy

    n_p = D_x.shape[1]  # zgarniamy długość wektora rozwiązań ciśnienia
    sol = np.zeros([n_u_x + n_u_y + n_p, 2])  # inicjujemy macierz rozwiązań 
    b_dirichlet = np.zeros([n_u_x + n_u_y + n_p,1])  # wektor dla warunku brzegowego Dirichleta u = g_D
    
    print('Liczba równań: ', sol.shape[0]) 
    
    # Uwzględniamy warunki początkowe
    
    sol[:n_u_x + n_u_y, 0] = ksi_0
    for i in dirichlet_fun_nodes:
        sol[i,0] = dirichlet_function(P[i][1],dirichlet_function_option)
        sol[i + n_u_x, 0] = 0

    # Konstrukcja układu równań
    
    LHS = scipy.sparse.lil_matrix((n_u_x + n_u_y + n_p, n_u_x + n_u_y + n_p), dtype=float)
    RHS = np.zeros([n_u_x + n_u_y + n_p,1])
    
    zero_matrix = scipy.sparse.lil_matrix((n_u_x,n_u_x))  # macierze zer
    small_zero_matrix = scipy.sparse.lil_matrix((n_p,n_p))

    LHS1 = scipy.sparse.hstack([viscosity * A, zero_matrix, -D_x])  # uzupełniamy lewą stronę układu
    LHS2 = scipy.sparse.hstack([zero_matrix, viscosity * A, -D_y])
    LHS3 = scipy.sparse.hstack([D_x.T, D_y.T, small_zero_matrix])
    LHS = scipy.sparse.vstack([LHS1,LHS2,LHS3], format="lil")

    # Rozpatrujemy warunki brzegowe
    
    gamma_cols = np.arange(n_u_x + n_u_y,n_u_x + n_u_y + n_p,1)  # tworzymy listę kolumn, któych nie będziemy usuwać
    
    no_bound_x = no_boundary
    no_bound_y = [element + n_u_x for element in no_bound_x]
    no_bound = np.append(no_bound_x, no_bound_y)
    no_bound = np.append(no_bound, gamma_cols)

    sol_boundary = np.zeros([len(no_bound), 2])

    LHS = scipy.sparse.csr_matrix(LHS)
    
    # opcjonalne: 
    # c=0.0000000000001 # Poprawa uwarunkowania dla metody iteracyjnej
    # LHS = LHS + c * scipy.sparse.diags([1]*LHS.shape[1])
    
    for i in dirichlet_fun_nodes:  # obliczamy poprawkę wynikającą z warunku brzegowego Dirichleta u = g_D, na prawą stronę równania
        b_dirichlet = b_dirichlet + LHS[:,i] * dirichlet_function(P[i][1],dirichlet_function_option)

    LHS_bounded = LHS[no_bound,:]  # pomijamy niepotrzebne wiersze
    LHS_bounded = LHS_bounded[:,no_bound]  # pomijamy niepotrzebne kolumny

    RHS[:n_u_x] = b[:n_u_x]
    RHS[n_u_x:n_u_x + n_u_y] = b[n_u_x:]
    RHS_bounded = RHS - b_dirichlet
    RHS_bounded = RHS_bounded[no_bound]  # pomijamy niepotrzebne wiersze
    t2 = time.time()
    period[2] = period[2] + t2 - t1  # czas: konstrukcja układu równań cz.2
   
    plt.spy(LHS_bounded)
    plt.show()
    
    t1 = time.time()
    
    # Obliczenie rozwiązania
    
    sol_boundary[:, 1] = spsolve(LHS_bounded, RHS_bounded)  # rozwiązujemy solverem scipy
    # sol_boundary[:, 1], exitCode = gmres(LHS_bounded, RHS_bounded)  # rozwiązujemy GMRES
    # sol_boundary[:, 1] = pypardiso.spsolve(LHS_bounded, RHS_bounded)  # rozwiązujemy solverem pypardiso
    
    for j in range(len(sol[:,1])): # wyniki wstawiamy w odpowiednie miejsca rozwiązania
        sol[j,1] = np.NaN
    for j in dirichlet_fun_nodes:
        sol[j,1] = dirichlet_function(P[j][1],dirichlet_function_option)
        sol[j + n_u_x,1] = 0
    for j in dirichlet_zero_nodes:
        sol[j,1] = 0
        sol[j + n_u_x,1] = 0
    h = 0
    for j in range(len(sol[:,1])):
        if np.isnan(sol[j,1]):
            sol[j,1] = sol_boundary[h,1]
            h = h + 1
            
    t2 = time.time()
    period[2] = t2 - t1  # czas: rozwiązanie
        
    # Output
        
    ksi_x = sol[:n_u_x,:]
    ksi_y = sol[n_u_x:n_u_x + n_u_y,:]
    gamma = sol[n_u_x + n_u_y:,:]

    return ksi_x, ksi_y, gamma


In [ ]:
def euclidean_norm(a, b):
    """
        Funkcja obliczająca normę euklidesową wektorów z R^{2}.
        input: a,b - np.array(), wektory
        output: norma euklidesowa wektorów
    """
    
    return (a**2 + b**2)**(1/2)
